<h1>Churn Regression for a salary </h1>

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle


In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
#Encode categorical variables
label_encoder_gender =LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])

In [8]:
#Onehot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
#Combine one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis =1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
#split the data into features and targets
X = data.drop('EstimatedSalary', axis=1)
y= data['EstimatedSalary']


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

#Scale these feathures
scaler =StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
#save all the pickle file
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

<h1>Train ANN with regrassion problem</h1>


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
#Building the model
model = Sequential([
    Dense(64, activation='relu', input_shape= (X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #O/t layer for regression
])

#compile the model
model.compile(optimizer='adam', loss= 'mean_absolute_error',  metrics = ['mae'])

In [17]:
#summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir = "regression/logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
#setupEarly stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [22]:
#Train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs  = 100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 100359.4766 - mae: 100359.4766 - val_loss: 98490.2344 - val_mae: 98490.2344
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - loss: 99557.6875 - mae: 99557.6875 - val_loss: 96824.6328 - val_mae: 96824.6328
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 97246.8203 - mae: 97246.8203 - val_loss: 92600.2422 - val_mae: 92600.2422
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - loss: 92417.5391 - mae: 92417.5391 - val_loss: 85658.8672 - val_mae: 85658.8672
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step - loss: 84929.7969 - mae: 84929.7969 - val_loss: 76842.3672 - val_mae: 76842.3672
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - loss: 76716.7734 - mae: 76716.7734 - val_loss: 67833.7031 - val_mae: 67833.7031
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - loss: 66056.8594 - mae: 66056.8594 - val_loss: 60225.3750 - val_mae: 60225.3750
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:
#evaluate mode
%load_ext tensorboard

In [28]:
%tensorboard --logdir regression/logs/fit/

Reusing TensorBoard on port 6007 (pid 98988), started 0:00:17 ago. (Use '!kill 98988' to kill it.)

In [29]:
#Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 51071.8047 - mae: 51071.8047
Test MAE: 50191.4296875


In [30]:
model.save('regression_modle.h5')